In [1]:
import pandas as pd
#load the page level data and document level data for each document class (stored in different folders)
rel_page=pd.read_csv('Data/betadata-master/medical_records_release_form/pdf_text_by_page.csv',error_bad_lines=False,encoding='utf-8')
rel_page['class']='0'
rel_doc=pd.read_csv('Data/betadata-master/medical_records_release_form/pdf_text_by_file.csv',error_bad_lines=False,encoding='utf-8')
rel_doc['class']='0'
cons_page=pd.read_csv('Data/betadata-master/informed consent form/pdf_text_by_page.csv',error_bad_lines=False,encoding='utf-8')
cons_page['class']='1'
cons_doc=pd.read_csv('Data/betadata-master/informed consent form/pdf_text_by_file.csv',error_bad_lines=False,encoding='utf-8')
cons_doc['class']='1'
int_page=pd.read_csv('Data/betadata-master/patient intake form/pdf_text_by_page.csv',error_bad_lines=False,encoding='utf-8')
int_page['class']='2'
int_doc=pd.read_csv('Data/betadata-master/patient intake form/pdf_text_by_file.csv',error_bad_lines=False,encoding='utf-8')
int_doc['class']='2'
med_page=pd.read_csv('Data/betadata-master/medical form/pdf_text_by_page.csv',error_bad_lines=False,encoding='utf-8')
med_page['class']='3'
med_doc=pd.read_csv('Data/betadata-master/medical form/pdf_text_by_file.csv',error_bad_lines=False,encoding='utf-8')
med_doc['class']='3'
#print the dimensions
dataframes=[rel_page,rel_doc,cons_page,cons_doc,int_page,int_doc,med_page,med_doc]
for d in dataframes:
    print(d.shape)

(601, 8)
(306, 7)
(2135, 8)
(467, 7)
(2170, 8)
(488, 7)
(246, 8)
(87, 7)


In [2]:
doc_level=[rel_doc,cons_doc,int_doc,med_doc]
page_level=[rel_page,cons_page,int_page,med_page]
data_doc=pd.concat(doc_level)
data_page=pd.concat(page_level)
print(data_doc.shape) 
print(data_page.shape) 

(1348, 7)
(5152, 8)


In [3]:
#shuffle rows
data_doc = data_doc.sample(frac=1,random_state=3).reset_index(drop=True)
data_page= data_page.sample(frac=1,random_state=3).reset_index(drop=True)
print(data_doc['class'].value_counts(normalize=True))
print(data_doc['class'].value_counts())
print(data_page['class'].value_counts(normalize=True))
print(data_page['class'].value_counts())

2    0.362018
1    0.346439
0    0.227003
3    0.064540
Name: class, dtype: float64
2    488
1    467
0    306
3     87
Name: class, dtype: int64
2    0.421196
1    0.414402
0    0.116654
3    0.047748
Name: class, dtype: float64
2    2170
1    2135
0     601
3     246
Name: class, dtype: int64


In [4]:
#remove duplicates and NAs based on text content - expected to be a lot of these
data_doc=data_doc.drop_duplicates(subset=['text']).dropna(subset=['text'])
data_page=data_page.drop_duplicates(subset=['text']).dropna(subset=['text'])
#add an extra id column
data_doc['id'] = range(1, len(data_doc) + 1)
data_page['id'] = range(1, len(data_page) + 1)
print(data_doc.shape)
print(data_page.shape)

(951, 8)
(3630, 9)


In [5]:
# define X and y for each dataframe
Xd = data_doc.text
yd = data_doc['class']
Xp = data_page.text
yp = data_page['class']

In [6]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
#from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer

vect = CountVectorizer()

In [7]:
#need to replace ____ with whitespace
data_doc['text']=data_doc['text'].apply(lambda x: x.replace("_"," "))
data_page['text']=data_page['text'].apply(lambda x: x.replace("_"," "))
Xd=Xd.apply(lambda x: x.replace("_"," "))
Xp=Xp.apply(lambda x: x.replace("_"," "))
Xd_dtm=vect.fit_transform(Xd)
Xp_dtm=vect.fit_transform(Xp)
print(Xd_dtm.shape)
print(Xp_dtm.shape)
print(vect.get_feature_names()[5900:12000])

(951, 38491)
(3630, 37709)
['animation', 'anindividual', 'aninformation', 'aninformed', 'aninvestigatormustinformtheirbiftheyintend', 'anit', 'ankle', 'ankles', 'ankylosing', 'ankylosis', 'anmed', 'ann', 'anna', 'annals', 'annas', 'anne', 'anniversary', 'annotated', 'annotations', 'announce', 'announced', 'announcements', 'announces', 'annoy', 'annoyed', 'annoying', 'annoys', 'annu', 'annual', 'annually', 'annualstatutes', 'ano', 'anomalies', 'anomaly', 'anon', 'anonymity', 'anonymization', 'anonymous', 'anonymously', 'anopportunitytoaskanyquestions', 'anorectices', 'anorexia', 'anoscopy', 'anot', 'anote', 'another', 'anotherstudy', 'anp', 'anprovide', 'anpualrrquorssuch', 'anr', 'anreosbtsiaem', 'ans', 'ansara', 'anselmo', 'ansometimes', 'answ', 'answe', 'answer', 'answercompletedby', 'answered', 'answeredwithina', 'answerfield', 'answergiventorequestorby', 'answerinatertiary', 'answering', 'answerisneededhelpsprioritizerequestsand', 'answerneededfield', 'answers', 'answersandmtmlt', 

In [8]:
#import all necessary libraries
from time import time
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report as clsr
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
import numpy as np
import tensorflow as tf
#training sets
Xd_train, Xd_test, yd_train, yd_test= train_test_split(Xd, yd, train_size=0.75, random_state=3)
Xp_train, Xp_test, yp_train, yp_test= train_test_split(Xp, yp, train_size=0.75, random_state=3)

/home/sierra/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/sierra/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


## Document Level Data

### using CNN - adapting from this example: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/examples/learn/text_classification_cnn.py

In [9]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#   http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import sys

from sklearn import metrics

learn = tf.contrib.learn

def cnn_model(features, target):
    """2 layer ConvNet to predict from sequence of words to a class."""
    # Convert indexes of words into embeddings.
    # This creates embeddings matrix of [n_words, EMBEDDING_SIZE] and then
    # maps word indexes of the sequence into [batch_size, sequence_length,
    # EMBEDDING_SIZE].
    target = tf.one_hot(target, 4, 1, 0)
    word_vectors = tf.contrib.layers.embed_sequence(
      features, vocab_size=n_words, embed_dim=EMBEDDING_SIZE, scope='words')
    word_vectors = tf.expand_dims(word_vectors, 3)
    with tf.variable_scope('CNN_Layer1'):
        conv1 = tf.contrib.layers.convolution2d(
            word_vectors, N_FILTERS, FILTER_SHAPE1, padding='VALID')
        # Add a RELU for non linearity.
        conv1 = tf.nn.relu(conv1)
        # Max pooling across output of Convolution+Relu.
        pool1 = tf.nn.max_pool(
            conv1,
            ksize=[1, POOLING_WINDOW, 1, 1],
            strides=[1, POOLING_STRIDE, 1, 1],
            padding='SAME')
        # Transpose matrix so that n_filters from convolution becomes width.
        pool1 = tf.transpose(pool1, [0, 1, 3, 2])
    with tf.variable_scope('CNN_Layer2'):
    # Second level of convolution filtering.
        conv2 = tf.contrib.layers.convolution2d(
        pool1, N_FILTERS, FILTER_SHAPE2, padding='VALID')
        # Max across each filter to get useful features for classification.
        pool2 = tf.squeeze(tf.reduce_max(conv2, 1), squeeze_dims=[1])

    # Apply regular WX + B and classification.
    logits = tf.contrib.layers.fully_connected(pool2, 4, activation_fn=None)
    loss = tf.contrib.losses.softmax_cross_entropy(logits, target)

    train_op = tf.contrib.layers.optimize_loss(
    loss,
    tf.contrib.framework.get_global_step(),
    optimizer='Adam',
    learning_rate=0.01)

    return ({
    'class': tf.argmax(logits, 1),
    'prob': tf.nn.softmax(logits)
    }, loss, train_op)

In [9]:
FLAGS = None

MAX_DOCUMENT_LENGTH = 5000
EMBEDDING_SIZE = 50
N_FILTERS = 10
WINDOW_SIZE = 20
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
POOLING_WINDOW = 4
POOLING_STRIDE = 2
n_words = 0


def main(unused_argv):
    global n_words
    # Prepare training and testing data
    x_train = Xd_train
    y_train = yd_train.astype('int')
    x_test = Xd_test
    y_test = yd_test.astype('int')

    # Process vocabulary
    vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    x_train = np.array(list(vocab_processor.fit_transform(x_train)))
    x_test = np.array(list(vocab_processor.transform(x_test)))
    n_words = len(vocab_processor.vocabulary_)
    print('Total words: %d' % n_words)

    # Build model
    classifier = learn.Estimator(model_fn=cnn_model)

    # Train and predict
    classifier.fit(x_train, y_train, steps=100)
    y_predicted = [
      p['class'] for p in classifier.predict(
          x_test, as_iterable=True)
    ]
    score = metrics.accuracy_score(y_test, y_predicted)
    print('Accuracy: {0:f}'.format(score))
    print(clsr(y_test, y_predicted))
    print(confusion_matrix(y_test,y_predicted))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--test_with_fake_data',
      default=False,
      help='Test the example code with fake data.',
      action='store_true')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Total words: 41660
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f5b26c6d828>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKComp

/home/sierra/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:loss = 1.38587, step = 1
INFO:tensorflow:Saving checkpoints for 7 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 13 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 19 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 25 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 30 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 36 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 42 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpoints for 48 into /tmp/tmpuwalbjna/model.ckpt.
INFO:tensorflow:Saving checkpo

SystemExit: 

/home/sierra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Page Level Data

In [10]:
FLAGS = None

MAX_DOCUMENT_LENGTH = 1000 #made slightly less
EMBEDDING_SIZE = 50
N_FILTERS = 10
WINDOW_SIZE = 20
FILTER_SHAPE1 = [WINDOW_SIZE, EMBEDDING_SIZE]
FILTER_SHAPE2 = [WINDOW_SIZE, N_FILTERS]
POOLING_WINDOW = 4
POOLING_STRIDE = 2
n_words = 0

def main(unused_argv):
    global n_words
    # Prepare training and testing data
    x_train = Xp_train
    y_train = yp_train.astype('int')
    x_test = Xp_test
    y_test = yp_test.astype('int')

    # Process vocabulary
    vocab_processor = learn.preprocessing.VocabularyProcessor(MAX_DOCUMENT_LENGTH)
    x_train = np.array(list(vocab_processor.fit_transform(x_train)))
    x_test = np.array(list(vocab_processor.transform(x_test)))
    n_words = len(vocab_processor.vocabulary_)
    print('Total words: %d' % n_words)

    # Build model
    classifier = learn.Estimator(model_fn=cnn_model)

    # Train and predict
    classifier.fit(x_train, y_train, steps=100)
    y_predicted = [
      p['class'] for p in classifier.predict(
          x_test, as_iterable=True)
    ]
    score = metrics.accuracy_score(y_test, y_predicted)
    print('Accuracy: {0:f}'.format(score))
    print(clsr(y_test, y_predicted))
    print(confusion_matrix(y_test,y_predicted))

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument(
      '--test_with_fake_data',
      default=False,
      help='Test the example code with fake data.',
      action='store_true')
    FLAGS, unparsed = parser.parse_known_args()
    tf.app.run(main=main, argv=[sys.argv[0]] + unparsed)

Total words: 44293
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f18d8112f28>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKComp

/home/sierra/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/deprecation.py:247: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.
  equality = a == b


Instructions for updating:
Use tf.losses.softmax_cross_entropy instead.
Instructions for updating:
Use tf.losses.compute_weighted_loss instead.
Instructions for updating:
Use tf.losses.add_loss instead.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:loss = 1.38703, step = 1
INFO:tensorflow:Saving checkpoints for 10 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 19 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 28 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 37 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 46 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 55 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 63 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkpoints for 71 into /tmp/tmpt4f38vab/model.ckpt.
INFO:tensorflow:Saving checkp

SystemExit: 

/home/sierra/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2889: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
